In [18]:
import os
import sys

sys.path.append("../src")  # for src imports
import geopandas as gpd
from owslib.wms import WebMapService

from wms.get_wms_crops import get_crop_bboxes

In [19]:
source_geometries_file = r"..\data\de_wildlife_crossing.geojson"
output_folder = r"..\data\de_wms_export_v2"
wms_url = "https://image.discomap.eea.europa.eu/arcgis/services/GioLand/VHR_2021_LAEA/ImageServer/WMSServer/?request=GetCapabilities&service=WMS"
pixel_size = 2  # meter per pixel
buffer_size = 100  # meter

In [20]:
gdf = gpd.read_file(r"..\data\de_wildlife_crossing.geojson")
gdf = gdf[gdf["bridge"] == "yes"]

In [21]:
wms = WebMapService(
    wms_url,
    version="1.3.0",
)

In [22]:
layer_name = list(wms.contents)[0]
print(layer_name)

VHR_2021_LAEA


In [ ]:
gdf_centroids = gdf.to_crs(epsg=3056).centroid
areas_of_interest = get_crop_bboxes(gdf_centroids, buffer_size)

0    POINT (2513788.313 6084033.962)
3    POINT (2186781.301 5929118.786)
4     POINT (2235677.73 6106661.762)
5    POINT (2229919.422 6285143.789)
6    POINT (2237322.472 6110635.813)
dtype: geometry

In [24]:
# for every point create a 200x200m polygon around it and request a WMS crop centered on the point
img_format = "image/png"
img_size = (
    int(buffer_size * 2 / pixel_size),
    int(buffer_size * 2 / pixel_size),
)  # width, height in pixels

for idx, bbox in enumerate(areas_of_interest):
    # Request WMS image
    img = wms.getmap(
        layers=[layer_name],
        srs="EPSG:3056",
        bbox=bbox,
        size=img_size,
        format=img_format,
        transparent=True,
    )

    # Save image to file
    img_data = img.read()
    img_filename = os.path.join(output_folder, f"crossing_{idx}.png")
    with open(img_filename, "wb") as f:
        f.write(img_data)

    print(f"Saved WMS crop for crossing {idx} to {img_filename}")

Saved WMS crop for crossing 0 to ..\data\de_wms_export_v2\crossing_0.png
Saved WMS crop for crossing 1 to ..\data\de_wms_export_v2\crossing_1.png
Saved WMS crop for crossing 2 to ..\data\de_wms_export_v2\crossing_2.png
Saved WMS crop for crossing 3 to ..\data\de_wms_export_v2\crossing_3.png
Saved WMS crop for crossing 4 to ..\data\de_wms_export_v2\crossing_4.png


KeyboardInterrupt: 